In [81]:
import tensorflow as tf

In [82]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pickle
import time

model_name = "ConvNet-64x2-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir="./analyzing_models/{}".format(model_name))

x = pickle.load(open('X.pickle',"rb"))
y = pickle.load(open('y.pickle',"rb"))


In [83]:
x = x/255.0

In [84]:
model = Sequential()

In [85]:
model.add(Conv2D(64, (5,5), input_shape = x.shape[1:],padding='Same',kernel_initializer="he_normal",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation("tanh"))

model.add(Dense(1))
model.add(Activation(activation="sigmoid"))

In [86]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics = ["accuracy"])

In [87]:
model.fit(x,y,batch_size=64,validation_split=0.1,callbacks=[tensorboard],epochs=2)

Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 90s 10ms/step - loss: 0.7407 - acc: 0.5134 - val_loss: 0.6933 - val_acc: 0.5050
Epoch 2/2
9000/9000 [==============================] - 85s 9ms/step - loss: 0.6944 - acc: 0.5013 - val_loss: 0.6872 - val_acc: 0.5620


In [88]:
model.save("Conv.model")

In [28]:
conv = tf.keras.models.load_model("Conv.model")

In [29]:
conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
activation_4 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
activation_5 (Activation)    (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7744)              0         
__________

In [30]:
pred = model.predict(x)

In [31]:
pred.shape

(10000, 1)

In [32]:
import numpy as np

In [33]:
predictions = np.zeros((x.shape[0],1))
for i in range(pred.shape[0]):
    predictions[i,0] = 1 if pred[i,0]>0.5 else 0

In [34]:
true_labels = y.reshape(y.shape[0],1)

In [35]:
accuracy = 100 - np.mean(np.abs(predictions-true_labels))*100

In [36]:
print("The accuracy of the model is: %.2f"%(accuracy))

The accuracy of the model is: 68.68
